# Agent Development Environment (ADE) for Healthcare Data Documentation

## Version 3.0

This notebook implements a specialized development environment for building, testing, and managing AI agents using the Google Gemini API and Agent Development Kit (ADK).

### System Overview

The ADE creates an "Orchestrator" and team of sub-agents that:
- Ingest complex, technical, and often imperfect data specifications (CSV, XML, JSON)
- Produce comprehensive, human-readable documentation
- Provide Human-in-the-Loop (HITL) review workflows
- Manage context using the "Toon" notation system

### Key Components

1. **SQLite Database** - Project-local persistence
2. **Toon System** - Context management for large files
3. **Core Agents** - Specialized AI agents for data processing
4. **ReviewQueue** - Human-in-the-loop workflow
5. **Orchestrator** - Agent chaining and workflow management

## 1. Setup and Dependencies

In [ ]:
# Install required packages
!pip install -q google-generativeai sqlite3 pandas numpy

In [ ]:
import sqlite3
import json
import pandas as pd
import numpy as np
from datetime import datetime
from typing import Dict, List, Optional, Any, Tuple
from enum import Enum
import google.generativeai as genai
from dataclasses import dataclass, asdict
import hashlib
import os
import time

In [ ]:
# Configure Gemini API
# Note: In Kaggle, add your API key as a secret named 'GOOGLE_API_KEY'
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
GEMINI_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

### ⚠️ Important: API Rate Limits

**Gemini API Free Tier Limits:**
- **10 requests per minute** per model
- The system automatically handles rate limiting with delays between requests
- Each API call will wait ~6 seconds to stay within limits
- Processing 7 variables takes approximately **2-3 minutes** due to rate limiting

**Tips for Faster Processing:**
1. Use `auto_approve=True` for testing (skips manual review)
2. Start with a small dataset (3-5 variables) to test
3. For production with paid tier, limits are much higher
4. The system automatically retries with exponential backoff if limits are hit

**If you see quota errors:**
- The system will automatically retry (up to 3 times)
- Wait times increase with each retry (6s, 12s, 24s)
- Consider upgrading to paid tier for higher quotas

## 2. Database Schema and Setup

The SQLite database is the backbone of the HITL workflow and provides persistent storage for all project data.

In [ ]:
class DatabaseManager:
    """Manages SQLite database operations for the ADE project."""
    
    def __init__(self, db_path: str = "project.db"):
        self.db_path = db_path
        self.conn = None
        self.cursor = None
        
    def connect(self):
        """Establish database connection."""
        self.conn = sqlite3.connect(self.db_path)
        self.conn.row_factory = sqlite3.Row
        self.cursor = self.conn.cursor()
        
    def close(self):
        """Close database connection."""
        if self.conn:
            self.conn.close()
            
    def initialize_schema(self):
        """Create all required tables for the ADE system."""
        
        # Agents table
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS Agents (
            agent_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL UNIQUE,
            system_prompt TEXT NOT NULL,
            agent_type TEXT NOT NULL,
            config JSON,
            created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
            updated_at DATETIME DEFAULT CURRENT_TIMESTAMP
        )
        """)
        
        # Toons table - Context snippets
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS Toons (
            toon_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL UNIQUE,
            toon_type TEXT NOT NULL CHECK(toon_type IN (
                'Toon_Summary', 'Toon_Chunk', 'Toon_Instruction', 
                'Toon_Version', 'Toon_Design', 'Toon_Mapping'
            )),
            content TEXT NOT NULL,
            metadata JSON,
            created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
            updated_at DATETIME DEFAULT CURRENT_TIMESTAMP
        )
        """)
        
        # Jobs table
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS Jobs (
            job_id TEXT PRIMARY KEY,
            source_file TEXT NOT NULL,
            status TEXT NOT NULL DEFAULT 'Running' CHECK(status IN (
                'Running', 'Completed', 'Failed', 'Paused'
            )),
            metadata JSON,
            created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
            updated_at DATETIME DEFAULT CURRENT_TIMESTAMP
        )
        """)
        
        # ReviewQueue table - HITL workflow
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS ReviewQueue (
            item_id INTEGER PRIMARY KEY AUTOINCREMENT,
            job_id TEXT NOT NULL,
            status TEXT NOT NULL DEFAULT 'Pending' CHECK(status IN (
                'Pending', 'Approved', 'Rejected', 'Needs_Clarification'
            )),
            source_agent TEXT NOT NULL,
            target_agent TEXT,
            source_data TEXT NOT NULL,
            generated_content TEXT NOT NULL,
            approved_content TEXT,
            rejection_feedback TEXT,
            clarification_response TEXT,
            created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
            updated_at DATETIME DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (job_id) REFERENCES Jobs(job_id)
        )
        """)
        
        # SystemState table - Application state
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS SystemState (
            state_key TEXT PRIMARY KEY,
            state_value TEXT NOT NULL,
            updated_at DATETIME DEFAULT CURRENT_TIMESTAMP
        )
        """)
        
        # SessionHistory table - Chat logs
        self.cursor.execute("""
        CREATE TABLE IF NOT EXISTS SessionHistory (
            history_id INTEGER PRIMARY KEY AUTOINCREMENT,
            job_id TEXT,
            role TEXT NOT NULL CHECK(role IN ('user', 'assistant', 'system')),
            content TEXT NOT NULL,
            metadata JSON,
            created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
            FOREIGN KEY (job_id) REFERENCES Jobs(job_id)
        )
        """)
        
        self.conn.commit()
        print("Database schema initialized successfully.")
        
    def execute_query(self, query: str, params: tuple = ()):
        """Execute a query and return results."""
        self.cursor.execute(query, params)
        return self.cursor.fetchall()
    
    def execute_update(self, query: str, params: tuple = ()):
        """Execute an update/insert query."""
        self.cursor.execute(query, params)
        self.conn.commit()
        return self.cursor.lastrowid

In [ ]:
# Initialize database
db = DatabaseManager("project.db")
db.connect()
db.initialize_schema()

## 3. Toon System - Context Management

The "Toon" system is a critical component for managing large files and providing granular control over agent context. A Toon is a discrete, named snippet of context stored in the database.

In [ ]:
class ToonType(Enum):
    """Enumeration of Toon types."""
    SUMMARY = "Toon_Summary"
    CHUNK = "Toon_Chunk"
    INSTRUCTION = "Toon_Instruction"
    VERSION = "Toon_Version"
    DESIGN = "Toon_Design"
    MAPPING = "Toon_Mapping"


@dataclass
class Toon:
    """Represents a context snippet (Toon)."""
    name: str
    toon_type: ToonType
    content: str
    metadata: Optional[Dict[str, Any]] = None
    toon_id: Optional[int] = None
    

class ToonManager:
    """Manages the Toon Library for context management."""
    
    def __init__(self, db_manager: DatabaseManager):
        self.db = db_manager
        
    def create_toon(self, name: str, toon_type: ToonType, content: str, 
                    metadata: Optional[Dict] = None) -> int:
        """Create a new Toon in the library."""
        query = """
        INSERT INTO Toons (name, toon_type, content, metadata)
        VALUES (?, ?, ?, ?)
        """
        metadata_json = json.dumps(metadata) if metadata else None
        toon_id = self.db.execute_update(
            query, 
            (name, toon_type.value, content, metadata_json)
        )
        print(f"Created Toon '{name}' (ID: {toon_id}, Type: {toon_type.value})")
        return toon_id
    
    def get_toon(self, toon_id: int) -> Optional[Toon]:
        """Retrieve a Toon by ID."""
        query = "SELECT * FROM Toons WHERE toon_id = ?"
        result = self.db.execute_query(query, (toon_id,))
        if result:
            row = result[0]
            return Toon(
                toon_id=row['toon_id'],
                name=row['name'],
                toon_type=ToonType(row['toon_type']),
                content=row['content'],
                metadata=json.loads(row['metadata']) if row['metadata'] else None
            )
        return None
    
    def get_toon_by_name(self, name: str) -> Optional[Toon]:
        """Retrieve a Toon by name."""
        query = "SELECT * FROM Toons WHERE name = ?"
        result = self.db.execute_query(query, (name,))
        if result:
            row = result[0]
            return Toon(
                toon_id=row['toon_id'],
                name=row['name'],
                toon_type=ToonType(row['toon_type']),
                content=row['content'],
                metadata=json.loads(row['metadata']) if row['metadata'] else None
            )
        return None
    
    def list_toons(self, toon_type: Optional[ToonType] = None) -> List[Toon]:
        """List all Toons, optionally filtered by type."""
        if toon_type:
            query = "SELECT * FROM Toons WHERE toon_type = ? ORDER BY created_at DESC"
            results = self.db.execute_query(query, (toon_type.value,))
        else:
            query = "SELECT * FROM Toons ORDER BY created_at DESC"
            results = self.db.execute_query(query)
        
        toons = []
        for row in results:
            toons.append(Toon(
                toon_id=row['toon_id'],
                name=row['name'],
                toon_type=ToonType(row['toon_type']),
                content=row['content'],
                metadata=json.loads(row['metadata']) if row['metadata'] else None
            ))
        return toons
    
    def update_toon(self, toon_id: int, content: Optional[str] = None,
                    metadata: Optional[Dict] = None):
        """Update a Toon's content or metadata."""
        updates = []
        params = []
        
        if content is not None:
            updates.append("content = ?")
            params.append(content)
        
        if metadata is not None:
            updates.append("metadata = ?")
            params.append(json.dumps(metadata))
        
        if updates:
            updates.append("updated_at = CURRENT_TIMESTAMP")
            query = f"UPDATE Toons SET {', '.join(updates)} WHERE toon_id = ?"
            params.append(toon_id)
            self.db.execute_update(query, tuple(params))
            print(f"Updated Toon ID {toon_id}")
    
    def delete_toon(self, toon_id: int):
        """Delete a Toon from the library."""
        query = "DELETE FROM Toons WHERE toon_id = ?"
        self.db.execute_update(query, (toon_id,))
        print(f"Deleted Toon ID {toon_id}")

## 4. Review Queue - Human-in-the-Loop Workflow

The ReviewQueue manages the HITL workflow, allowing human reviewers to approve, reject, or request clarification on agent-generated content.

In [ ]:
class ReviewStatus(Enum):
    """Status values for review items."""
    PENDING = "Pending"
    APPROVED = "Approved"
    REJECTED = "Rejected"
    NEEDS_CLARIFICATION = "Needs_Clarification"


@dataclass
class ReviewItem:
    """Represents an item in the review queue."""
    job_id: str
    source_agent: str
    source_data: str
    generated_content: str
    status: ReviewStatus = ReviewStatus.PENDING
    target_agent: Optional[str] = None
    approved_content: Optional[str] = None
    rejection_feedback: Optional[str] = None
    clarification_response: Optional[str] = None
    item_id: Optional[int] = None


class ReviewQueueManager:
    """Manages the review queue for HITL workflows."""
    
    def __init__(self, db_manager: DatabaseManager):
        self.db = db_manager
    
    def add_item(self, item: ReviewItem) -> int:
        """Add an item to the review queue."""
        query = """
        INSERT INTO ReviewQueue 
        (job_id, status, source_agent, target_agent, source_data, generated_content)
        VALUES (?, ?, ?, ?, ?, ?)
        """
        item_id = self.db.execute_update(
            query,
            (item.job_id, item.status.value, item.source_agent, 
             item.target_agent, item.source_data, item.generated_content)
        )
        print(f"Added item {item_id} to review queue (Status: {item.status.value})")
        return item_id
    
    def get_pending_items(self, job_id: Optional[str] = None) -> List[ReviewItem]:
        """Get all pending review items, optionally filtered by job_id."""
        if job_id:
            query = "SELECT * FROM ReviewQueue WHERE status = 'Pending' AND job_id = ?"
            results = self.db.execute_query(query, (job_id,))
        else:
            query = "SELECT * FROM ReviewQueue WHERE status = 'Pending'"
            results = self.db.execute_query(query)
        
        return [self._row_to_item(row) for row in results]
    
    def get_clarification_items(self, job_id: Optional[str] = None) -> List[ReviewItem]:
        """Get items needing clarification."""
        if job_id:
            query = "SELECT * FROM ReviewQueue WHERE status = 'Needs_Clarification' AND job_id = ?"
            results = self.db.execute_query(query, (job_id,))
        else:
            query = "SELECT * FROM ReviewQueue WHERE status = 'Needs_Clarification'"
            results = self.db.execute_query(query)
        
        return [self._row_to_item(row) for row in results]
    
    def approve_item(self, item_id: int, approved_content: Optional[str] = None):
        """Approve a review item, optionally with edited content."""
        query = """
        UPDATE ReviewQueue 
        SET status = 'Approved', approved_content = ?, updated_at = CURRENT_TIMESTAMP
        WHERE item_id = ?
        """
        # If no approved content provided, use the generated content
        if approved_content is None:
            item = self.get_item(item_id)
            approved_content = item.generated_content
        
        self.db.execute_update(query, (approved_content, item_id))
        print(f"Approved item {item_id}")
    
    def reject_item(self, item_id: int, feedback: str):
        """Reject a review item with feedback."""
        query = """
        UPDATE ReviewQueue 
        SET status = 'Rejected', rejection_feedback = ?, updated_at = CURRENT_TIMESTAMP
        WHERE item_id = ?
        """
        self.db.execute_update(query, (feedback, item_id))
        print(f"Rejected item {item_id}")
    
    def request_clarification(self, item_id: int, question: str):
        """Mark item as needing clarification."""
        query = """
        UPDATE ReviewQueue 
        SET status = 'Needs_Clarification', generated_content = ?, updated_at = CURRENT_TIMESTAMP
        WHERE item_id = ?
        """
        self.db.execute_update(query, (question, item_id))
        print(f"Requested clarification for item {item_id}")
    
    def submit_clarification(self, item_id: int, response: str):
        """Submit clarification response and return to pending."""
        query = """
        UPDATE ReviewQueue 
        SET clarification_response = ?, status = 'Pending', updated_at = CURRENT_TIMESTAMP
        WHERE item_id = ?
        """
        self.db.execute_update(query, (response, item_id))
        print(f"Submitted clarification for item {item_id}")
    
    def get_item(self, item_id: int) -> Optional[ReviewItem]:
        """Get a specific review item."""
        query = "SELECT * FROM ReviewQueue WHERE item_id = ?"
        results = self.db.execute_query(query, (item_id,))
        if results:
            return self._row_to_item(results[0])
        return None
    
    def get_approved_items(self, job_id: str) -> List[ReviewItem]:
        """Get all approved items for a job."""
        query = "SELECT * FROM ReviewQueue WHERE job_id = ? AND status = 'Approved'"
        results = self.db.execute_query(query, (job_id,))
        return [self._row_to_item(row) for row in results]
    
    def _row_to_item(self, row) -> ReviewItem:
        """Convert database row to ReviewItem."""
        return ReviewItem(
            item_id=row['item_id'],
            job_id=row['job_id'],
            status=ReviewStatus(row['status']),
            source_agent=row['source_agent'],
            target_agent=row['target_agent'],
            source_data=row['source_data'],
            generated_content=row['generated_content'],
            approved_content=row['approved_content'],
            rejection_feedback=row['rejection_feedback'],
            clarification_response=row['clarification_response']
        )

## 5. Core Agent Classes

These agents form the processing pipeline for healthcare data documentation.

In [ ]:
class BaseAgent:
    """Base class for all agents with rate limiting and retry logic."""
    
    def __init__(self, name: str, system_prompt: str, 
                 model_name: str = "gemini-2.0-flash-exp",
                 requests_per_minute: int = 10):
        self.name = name
        self.system_prompt = system_prompt
        self.model_name = model_name
        self.model = genai.GenerativeModel(model_name)
        self.active_toons: List[Toon] = []
        self.requests_per_minute = requests_per_minute
        self.min_delay = 60.0 / requests_per_minute  # Minimum delay between requests
        self.last_request_time = 0
        
    def inject_toons(self, toons: List[Toon]):
        """Inject Toons into agent context."""
        self.active_toons = toons
        
    def build_prompt(self, user_input: str, additional_context: str = "") -> str:
        """Build the full prompt with system prompt, Toons, and user input."""
        prompt_parts = [self.system_prompt]
        
        # Add active Toons as context
        if self.active_toons:
            prompt_parts.append("\n=== CONTEXT (Toons) ===")
            for toon in self.active_toons:
                prompt_parts.append(f"\n[{toon.toon_type.value}: {toon.name}]")
                prompt_parts.append(toon.content)
        
        # Add additional context
        if additional_context:
            prompt_parts.append("\n=== ADDITIONAL CONTEXT ===")
            prompt_parts.append(additional_context)
        
        # Add user input
        prompt_parts.append("\n=== INPUT ===")
        prompt_parts.append(user_input)
        
        return "\n".join(prompt_parts)
    
    def _wait_for_rate_limit(self):
        """Implement rate limiting by waiting if necessary."""
        if self.last_request_time > 0:
            elapsed = time.time() - self.last_request_time
            if elapsed < self.min_delay:
                wait_time = self.min_delay - elapsed
                print(f"⏱️  Rate limiting: waiting {wait_time:.1f}s...")
                time.sleep(wait_time)
    
    def generate(self, prompt: str, max_retries: int = 3) -> str:
        """Generate response using Gemini API with retry logic and rate limiting."""
        for attempt in range(max_retries):
            try:
                # Wait for rate limit before making request
                self._wait_for_rate_limit()
                
                # Make API call
                self.last_request_time = time.time()
                response = self.model.generate_content(prompt)
                return response.text
                
            except Exception as e:
                error_str = str(e)
                
                # Check if it's a rate limit error
                if "ResourceExhausted" in error_str or "429" in error_str:
                    # Extract retry delay from error if available
                    retry_delay = 6.0  # Default to 6 seconds
                    
                    # Try to extract the suggested retry delay
                    if "retry_delay" in error_str:
                        try:
                            import re
                            match = re.search(r'seconds: (\d+)', error_str)
                            if match:
                                retry_delay = float(match.group(1))
                        except:
                            pass
                    
                    # Add exponential backoff
                    wait_time = retry_delay * (2 ** attempt)
                    
                    if attempt < max_retries - 1:
                        print(f"⚠️  Rate limit hit. Waiting {wait_time:.1f}s before retry {attempt + 1}/{max_retries}...")
                        time.sleep(wait_time)
                    else:
                        print(f"❌ Max retries reached. Error: {error_str}")
                        raise
                else:
                    # For non-rate-limit errors, raise immediately
                    print(f"❌ API Error: {error_str}")
                    raise
        
        raise Exception(f"Failed after {max_retries} retries")
    
    def process(self, input_data: str, additional_context: str = "") -> str:
        """Process input and return output."""
        full_prompt = self.build_prompt(input_data, additional_context)
        return self.generate(full_prompt)

In [ ]:
class DataParserAgent(BaseAgent):
    """Agent for parsing raw data into standardized JSON format."""
    
    def __init__(self):
        system_prompt = """
You are a DataParserAgent specialized in converting raw data specifications 
(CSV, JSON, XML) into a standardized JSON format.

Your task:
1. Parse the input data
2. Preserve all original field names and values
3. Output a JSON array where each element represents one variable/field
4. Each element should include: original_name, original_type (if available), 
   original_description (if available), and any other metadata

Output format:
```json
[
  {
    "original_name": "field_name",
    "original_type": "type",
    "original_description": "description",
    "metadata": {}
  }
]
```

Only output valid JSON. No additional commentary.
"""
        super().__init__("DataParserAgent", system_prompt)
    
    def parse_csv(self, csv_data: str) -> List[Dict]:
        """Parse CSV data dictionary."""
        result = self.process(csv_data)
        # Extract JSON from markdown code block if present
        if "```json" in result:
            result = result.split("```json")[1].split("```")[0].strip()
        elif "```" in result:
            result = result.split("```")[1].split("```")[0].strip()
        return json.loads(result)

In [ ]:
class TechnicalAnalyzerAgent(BaseAgent):
    """Agent for analyzing technical properties and mapping to internal standards."""
    
    def __init__(self):
        system_prompt = """
You are a TechnicalAnalyzerAgent specialized in analyzing data fields and 
mapping them to internal standards.

Your task:
1. Analyze each field from the parsed data
2. Infer technical properties (data_type, constraints, cardinality)
3. Map to standardized field names following healthcare data conventions
4. If mapping is unclear or confidence is low, flag for clarification

Standard field mappings:
- variable_name: standardized variable name
- data_type: categorical, continuous, date, text, boolean
- description: human-readable description
- constraints: any validation rules
- cardinality: required, optional, repeated
- confidence: high, medium, low (for mapping quality)

Output format:
```json
[
  {
    "original_name": "field_name",
    "variable_name": "standardized_name",
    "data_type": "categorical",
    "description": "description",
    "constraints": {},
    "cardinality": "required",
    "confidence": "high",
    "needs_clarification": false,
    "clarification_question": ""
  }
]
```

Only output valid JSON. No additional commentary.
"""
        super().__init__("TechnicalAnalyzerAgent", system_prompt)
    
    def analyze(self, parsed_data: List[Dict], 
                clarifications: Optional[Dict[str, str]] = None) -> List[Dict]:
        """Analyze parsed data and map to internal standards."""
        additional_context = ""
        if clarifications:
            additional_context = "\n=== USER CLARIFICATIONS ===\n"
            for field, clarification in clarifications.items():
                additional_context += f"{field}: {clarification}\n"
        
        result = self.process(json.dumps(parsed_data, indent=2), additional_context)
        
        # Extract JSON from markdown code block if present
        if "```json" in result:
            result = result.split("```json")[1].split("```")[0].strip()
        elif "```" in result:
            result = result.split("```")[1].split("```")[0].strip()
        
        return json.loads(result)

In [ ]:
class DomainOntologyAgent(BaseAgent):
    """Agent for mapping to standard healthcare ontologies (OMOP, LOINC, etc.)."""
    
    def __init__(self):
        system_prompt = """
You are a DomainOntologyAgent specialized in mapping healthcare data fields 
to standard ontologies and terminologies.

Your task:
1. For each variable, identify appropriate standard ontology codes
2. Primary ontologies to consider:
   - OMOP CDM concepts
   - LOINC codes (for lab/clinical observations)
   - SNOMED CT (for clinical terms)
   - RxNorm (for medications)
3. Provide both the code and the standard term
4. Include a confidence score for each mapping

Output format:
```json
{
  "variable_name": "standardized_name",
  "ontology_mappings": [
    {
      "system": "OMOP",
      "code": "123456",
      "display": "Standard Concept Name",
      "confidence": "high"
    }
  ]
}
```

Only output valid JSON. No additional commentary.
"""
        super().__init__("DomainOntologyAgent", system_prompt)
    
    def map_ontologies(self, variable_data: Dict) -> Dict:
        """Map a variable to standard ontologies."""
        result = self.process(json.dumps(variable_data, indent=2))
        
        # Extract JSON from markdown code block if present
        if "```json" in result:
            result = result.split("```json")[1].split("```")[0].strip()
        elif "```" in result:
            result = result.split("```")[1].split("```")[0].strip()
        
        return json.loads(result)

In [ ]:
class PlainLanguageAgent(BaseAgent):
    """Agent for generating human-readable documentation."""
    
    def __init__(self):
        system_prompt = """
You are a PlainLanguageAgent specialized in creating clear, comprehensive, 
human-readable documentation for healthcare data variables.

Your task:
1. Convert technical variable specifications into plain language
2. Explain what the variable represents in clinical/research context
3. Describe data type, constraints, and valid values
4. Include ontology mappings and their significance
5. Write for an interdisciplinary audience (clinicians, researchers, data scientists)

Output format (Markdown):
```markdown
## Variable: [Variable Name]

**Description:** [Clear, concise description]

**Technical Details:**
- Data Type: [type]
- Cardinality: [required/optional]
- Valid Values: [constraints or ranges]

**Standard Ontology Mappings:**
- OMOP: [code] - [term]
- LOINC: [code] - [term]

**Clinical Context:** [Explanation of why this variable matters]
```

Only output Markdown documentation. No additional commentary.
"""
        super().__init__("PlainLanguageAgent", system_prompt)
    
    def document_variable(self, enriched_data: Dict) -> str:
        """Generate plain language documentation for a variable."""
        result = self.process(json.dumps(enriched_data, indent=2))
        
        # Remove markdown code block markers if present
        if "```markdown" in result:
            result = result.split("```markdown")[1].split("```")[0].strip()
        elif result.startswith("```") and result.endswith("```"):
            result = result.split("```")[1].split("```")[0].strip()
        
        return result

In [ ]:
class DocumentationAssemblerAgent(BaseAgent):
    """Agent for assembling final documentation from approved items."""
    
    def __init__(self, review_queue: ReviewQueueManager):
        system_prompt = """
You are a DocumentationAssemblerAgent specialized in creating comprehensive, 
well-structured data documentation.

Your task:
1. Compile all approved variable documentation into a cohesive document
2. Add a table of contents
3. Include metadata (generation date, source file, etc.)
4. Organize by logical groupings if applicable
5. Ensure consistent formatting throughout

Output: A complete Markdown document ready for publication.
"""
        super().__init__("DocumentationAssemblerAgent", system_prompt)
        self.review_queue = review_queue
    
    def assemble(self, job_id: str) -> str:
        """Assemble final documentation from approved review items."""
        approved_items = self.review_queue.get_approved_items(job_id)
        
        if not approved_items:
            return "# No approved documentation found for this job."
        
        # Build document
        doc_parts = [
            "# Healthcare Data Documentation",
            f"\n**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
            f"**Job ID:** {job_id}",
            "\n---\n"
        ]
        
        # Add table of contents
        doc_parts.append("## Table of Contents\n")
        for i, item in enumerate(approved_items, 1):
            # Extract variable name from content if possible
            content = item.approved_content
            if "## Variable:" in content:
                var_name = content.split("## Variable:")[1].split("\n")[0].strip()
                doc_parts.append(f"{i}. [{var_name}](#{var_name.lower().replace(' ', '-')})")
        
        doc_parts.append("\n---\n")
        
        # Add all approved content
        for item in approved_items:
            doc_parts.append(item.approved_content)
            doc_parts.append("\n---\n")
        
        return "\n".join(doc_parts)

## 6. Orchestrator - Agent Workflow Management

The Orchestrator manages the flow of data through the agent pipeline and handles the HITL workflow.

In [ ]:
class Orchestrator:
    """Manages the workflow of agents and coordinates the documentation pipeline."""
    
    def __init__(self, db_manager: DatabaseManager):
        self.db = db_manager
        self.toon_manager = ToonManager(db_manager)
        self.review_queue = ReviewQueueManager(db_manager)
        
        # Initialize agents
        self.data_parser = DataParserAgent()
        self.technical_analyzer = TechnicalAnalyzerAgent()
        self.domain_ontology = DomainOntologyAgent()
        self.plain_language = PlainLanguageAgent()
        self.assembler = DocumentationAssemblerAgent(self.review_queue)
        
    def create_job(self, source_file: str) -> str:
        """Create a new documentation job."""
        # Generate unique job ID
        job_id = hashlib.md5(
            f"{source_file}_{datetime.now().isoformat()}".encode()
        ).hexdigest()[:12]
        
        query = """
        INSERT INTO Jobs (job_id, source_file, status)
        VALUES (?, ?, 'Running')
        """
        self.db.execute_update(query, (job_id, source_file))
        print(f"Created job {job_id} for source file: {source_file}")
        return job_id
    
    def process_data_dictionary(self, source_data: str, source_file: str = "input.csv",
                                auto_approve: bool = False) -> str:
        """
        Main workflow: Process a data dictionary through the agent pipeline.
        
        Args:
            source_data: The raw data dictionary content
            source_file: Name of the source file
            auto_approve: If True, automatically approve all generated content
        
        Returns:
            job_id: The ID of the created job
        """
        # Create job
        job_id = self.create_job(source_file)
        
        print("\n=== Step 1: Parsing Data ===")
        parsed_data = self.data_parser.parse_csv(source_data)
        print(f"Parsed {len(parsed_data)} variables")
        
        print("\n=== Step 2: Technical Analysis ===")
        analyzed_data = self.technical_analyzer.analyze(parsed_data)
        print(f"Analyzed {len(analyzed_data)} variables")
        
        # Check for clarifications needed
        needs_clarification = [v for v in analyzed_data if v.get('needs_clarification', False)]
        if needs_clarification:
            print(f"\n⚠️  {len(needs_clarification)} variables need clarification")
            for var in needs_clarification:
                # Add to review queue
                item = ReviewItem(
                    job_id=job_id,
                    source_agent="TechnicalAnalyzerAgent",
                    source_data=json.dumps(var),
                    generated_content=var.get('clarification_question', 'Needs clarification'),
                    status=ReviewStatus.NEEDS_CLARIFICATION
                )
                self.review_queue.add_item(item)
        
        print("\n=== Step 3: Domain Ontology Mapping ===")
        enriched_data = []
        for i, var in enumerate(analyzed_data):
            if not var.get('needs_clarification', False):
                print(f"Mapping variable {i+1}/{len(analyzed_data)}: {var.get('variable_name', 'unknown')}")
                enriched = self.domain_ontology.map_ontologies(var)
                enriched_data.append(enriched)
        
        print(f"\nEnriched {len(enriched_data)} variables with ontology mappings")
        
        print("\n=== Step 4: Plain Language Documentation ===")
        for i, var in enumerate(enriched_data):
            print(f"Documenting variable {i+1}/{len(enriched_data)}: {var.get('variable_name', 'unknown')}")
            documentation = self.plain_language.document_variable(var)
            
            # Add to review queue
            item = ReviewItem(
                job_id=job_id,
                source_agent="PlainLanguageAgent",
                source_data=json.dumps(var),
                generated_content=documentation,
                status=ReviewStatus.PENDING
            )
            item_id = self.review_queue.add_item(item)
            
            # Auto-approve if requested
            if auto_approve:
                self.review_queue.approve_item(item_id)
        
        print(f"\n✓ Job {job_id} processing complete")
        print(f"  - {len(enriched_data)} items ready for review")
        if needs_clarification:
            print(f"  - {len(needs_clarification)} items need clarification")
        
        return job_id
    
    def finalize_documentation(self, job_id: str, output_file: str = "documentation.md") -> str:
        """Assemble and save final documentation."""
        print(f"\n=== Assembling Final Documentation ===")
        documentation = self.assembler.assemble(job_id)
        
        # Save to file
        with open(output_file, 'w') as f:
            f.write(documentation)
        
        print(f"✓ Documentation saved to {output_file}")
        
        # Update job status
        query = "UPDATE Jobs SET status = 'Completed' WHERE job_id = ?"
        self.db.execute_update(query, (job_id,))
        
        return documentation

## 7. Example Usage and Demonstration

Let's demonstrate the system with a sample healthcare data dictionary.

In [ ]:
# Sample REDCap-style data dictionary
sample_data_dictionary = """Variable Name,Field Type,Field Label,Choices,Notes
patient_id,text,Patient ID,,Unique identifier
age,integer,Age (years),,Age at enrollment
sex,radio,Biological Sex,"1, Male | 2, Female | 3, Other",
bp_systolic,integer,Systolic Blood Pressure (mmHg),,
bp_diastolic,integer,Diastolic Blood Pressure (mmHg),,
diagnosis_date,date,Diagnosis Date,,Date of primary diagnosis
hba1c,decimal,Hemoglobin A1c (%),,Glycated hemoglobin
"""

In [ ]:
# Initialize orchestrator
orchestrator = Orchestrator(db)

# Create some useful Toons for context
print("Creating Toons for context management...")

# Instruction Toon for OMOP mapping
orchestrator.toon_manager.create_toon(
    name="OMOP_Mapping_Instructions",
    toon_type=ToonType.INSTRUCTION,
    content="""When mapping to OMOP CDM:
- Blood pressure measurements should map to OMOP concept_id 3004249 (Systolic) and 3012888 (Diastolic)
- HbA1c should map to OMOP concept_id 3004410
- Age should be stored as an integer in years
- Sex should use standard OMOP gender concepts: 8507 (Male), 8532 (Female)
"""
)

# Design decision Toon
orchestrator.toon_manager.create_toon(
    name="Project_Design_Notes",
    toon_type=ToonType.DESIGN,
    content="""This is a diabetes research study collecting baseline clinical measurements.
All measurements follow standard clinical protocols. Blood pressure is measured in sitting position
after 5 minutes rest. HbA1c measured using DCCT-aligned assay.
"""
)

print("\nToons created successfully!")

In [ ]:
# Inject Toons into relevant agents
toons = orchestrator.toon_manager.list_toons()
orchestrator.domain_ontology.inject_toons(toons)
orchestrator.plain_language.inject_toons(toons)

print(f"Injected {len(toons)} Toons into agent context")

In [ ]:
# Process the data dictionary
job_id = orchestrator.process_data_dictionary(
    source_data=sample_data_dictionary,
    source_file="diabetes_study_data_dictionary.csv",
    auto_approve=True  # Set to False to enable manual review
)

In [ ]:
# Review pending items (if auto_approve=False)
pending_items = orchestrator.review_queue.get_pending_items(job_id)
print(f"\nPending review items: {len(pending_items)}")

if pending_items:
    print("\n=== Review Interface ===")
    for item in pending_items[:3]:  # Show first 3
        print(f"\nItem {item.item_id}:")
        print(f"Source: {item.source_agent}")
        print(f"\nGenerated Content:\n{item.generated_content[:500]}...")
        print("\nActions: [Approve], [Edit & Approve], [Reject]")
        print("-" * 80)

In [ ]:
# Example: Approve an item with edits
if pending_items:
    item_id = pending_items[0].item_id
    
    # Get the generated content
    original_content = pending_items[0].generated_content
    
    # Make an edit (example)
    edited_content = original_content.replace(
        "Clinical Context:",
        "Clinical Context: [EDITED BY REVIEWER]"
    )
    
    # Approve with edits
    orchestrator.review_queue.approve_item(item_id, edited_content)
    print(f"Approved item {item_id} with edits")

In [ ]:
# Check for clarification requests
clarification_items = orchestrator.review_queue.get_clarification_items(job_id)
print(f"\nItems needing clarification: {len(clarification_items)}")

if clarification_items:
    print("\n=== Clarification Interface ===")
    for item in clarification_items:
        print(f"\nItem {item.item_id}:")
        print(f"Question: {item.generated_content}")
        print("\nYour Response: [Enter clarification here]")
        print("-" * 80)

In [ ]:
# Example: Submit clarification
if clarification_items:
    item_id = clarification_items[0].item_id
    clarification_response = "This field represents the patient's biological sex as reported at enrollment."
    
    orchestrator.review_queue.submit_clarification(item_id, clarification_response)
    print(f"Submitted clarification for item {item_id}")

In [ ]:
# Finalize and generate documentation
final_documentation = orchestrator.finalize_documentation(
    job_id=job_id,
    output_file="healthcare_data_documentation.md"
)

print("\n=== Final Documentation Preview ===")
print(final_documentation[:1000] + "\n...")

## 8. Context Management - Working vs Long-Term Memory

Demonstration of the memory model and context compaction.

In [ ]:
class ContextManager:
    """Manages working and long-term memory for the ADE system."""
    
    def __init__(self, db_manager: DatabaseManager, max_tokens: int = 100000):
        self.db = db_manager
        self.max_tokens = max_tokens
        self.compaction_threshold = int(max_tokens * 0.8)
        
    def estimate_tokens(self, text: str) -> int:
        """Rough token estimation (1 token ≈ 4 characters)."""
        return len(text) // 4
    
    def get_working_memory(self, job_id: str) -> Dict[str, Any]:
        """Get current working memory for a job."""
        # Get active toons
        query = "SELECT state_value FROM SystemState WHERE state_key = ?"
        result = self.db.execute_query(query, (f"active_toons_{job_id}",))
        active_toon_ids = json.loads(result[0]['state_value']) if result else []
        
        # Get session history
        query = "SELECT * FROM SessionHistory WHERE job_id = ? ORDER BY created_at"
        history_rows = self.db.execute_query(query, (job_id,))
        
        session_history = []
        for row in history_rows:
            session_history.append({
                'role': row['role'],
                'content': row['content'],
                'timestamp': row['created_at']
            })
        
        # Calculate total tokens
        total_tokens = sum(self.estimate_tokens(msg['content']) for msg in session_history)
        
        return {
            'active_toon_ids': active_toon_ids,
            'session_history': session_history,
            'total_tokens': total_tokens,
            'needs_compaction': total_tokens > self.compaction_threshold
        }
    
    def add_to_session_history(self, job_id: str, role: str, content: str):
        """Add a message to session history."""
        query = """
        INSERT INTO SessionHistory (job_id, role, content)
        VALUES (?, ?, ?)
        """
        self.db.execute_update(query, (job_id, role, content))
    
    def compact_context(self, job_id: str) -> str:
        """Compact session history using summarization."""
        print("\n=== Context Compaction Triggered ===")
        
        working_memory = self.get_working_memory(job_id)
        session_history = working_memory['session_history']
        
        # Build conversation text
        conversation = "\n\n".join([
            f"{msg['role'].upper()}: {msg['content']}"
            for msg in session_history
        ])
        
        # Use Gemini to summarize
        compactor_prompt = f"""
You are a ContextCompactorAgent. Your task is to create a concise summary of this
conversation that preserves all critical information, decisions, and clarifications.

Conversation:
{conversation}

Provide a structured summary that includes:
1. Key decisions made
2. Important clarifications provided
3. Current state of the work

Summary:
"""
        
        model = genai.GenerativeModel("gemini-2.0-flash-exp")
        response = model.generate_content(compactor_prompt)
        summary = response.text
        
        print(f"Original tokens: {working_memory['total_tokens']}")
        print(f"Summary tokens: {self.estimate_tokens(summary)}")
        print(f"Reduction: {100 * (1 - self.estimate_tokens(summary) / working_memory['total_tokens']):.1f}%")
        
        # Store summary as a Toon
        toon_manager = ToonManager(self.db)
        toon_manager.create_toon(
            name=f"Session_Summary_{job_id}_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
            toon_type=ToonType.SUMMARY,
            content=summary,
            metadata={'job_id': job_id, 'original_tokens': working_memory['total_tokens']}
        )
        
        return summary
    
    def clear_context(self, job_id: str):
        """Clear working memory (flush session history and active toons)."""
        # Note: This doesn't delete from database, just resets the "active" state
        query = "UPDATE SystemState SET state_value = '[]' WHERE state_key = ?"
        self.db.execute_update(query, (f"active_toons_{job_id}",))
        
        print(f"Cleared working memory for job {job_id}")
        print("Note: Session history preserved in long-term memory (database)")

In [ ]:
# Demonstrate context management
context_manager = ContextManager(db)

# Simulate a conversation
context_manager.add_to_session_history(job_id, "user", "Please process the diabetes data dictionary")
context_manager.add_to_session_history(job_id, "assistant", "I'll process the data dictionary through the agent pipeline.")
context_manager.add_to_session_history(
    job_id, 
    "user", 
    "For the HbA1c variable, please note that values above 6.5% indicate diabetes diagnosis."
)

# Check working memory
working_memory = context_manager.get_working_memory(job_id)
print(f"\nWorking Memory Status:")
print(f"  Total tokens: {working_memory['total_tokens']}")
print(f"  Needs compaction: {working_memory['needs_compaction']}")
print(f"  Session messages: {len(working_memory['session_history'])}")

## 9. System Status and Monitoring

In [ ]:
def display_system_status(db: DatabaseManager):
    """Display current system status."""
    print("\n" + "="*80)
    print("ADE SYSTEM STATUS")
    print("="*80)
    
    # Jobs
    jobs = db.execute_query("SELECT * FROM Jobs")
    print(f"\nJobs: {len(jobs)}")
    for job in jobs:
        print(f"  [{job['job_id']}] {job['source_file']} - Status: {job['status']}")
    
    # Toons
    toons = db.execute_query("SELECT toon_type, COUNT(*) as count FROM Toons GROUP BY toon_type")
    print(f"\nToon Library:")
    for toon in toons:
        print(f"  {toon['toon_type']}: {toon['count']}")
    
    # Review Queue
    review_stats = db.execute_query(
        "SELECT status, COUNT(*) as count FROM ReviewQueue GROUP BY status"
    )
    print(f"\nReview Queue:")
    for stat in review_stats:
        print(f"  {stat['status']}: {stat['count']}")
    
    print("\n" + "="*80)

display_system_status(db)

## 10. Export and Cleanup

In [ ]:
# Export database for backup
import shutil

def backup_database(db_path: str, backup_path: str):
    """Create a backup of the project database."""
    shutil.copy2(db_path, backup_path)
    print(f"Database backed up to {backup_path}")

# Uncomment to create backup
# backup_database("project.db", "project_backup.db")

In [ ]:
# View generated documentation
if os.path.exists("healthcare_data_documentation.md"):
    with open("healthcare_data_documentation.md", 'r') as f:
        print("\n=== Generated Documentation ===")
        print(f.read())

## Summary

This notebook demonstrates a complete implementation of the Agent Development Environment (ADE) for Healthcare Data Documentation. Key features:

### Implemented Components:
1. **SQLite Database** - Full schema with all required tables
2. **Toon System** - Context management with 6 Toon types
3. **Review Queue** - Complete HITL workflow with status management
4. **Core Agents**:
   - DataParserAgent
   - TechnicalAnalyzerAgent
   - DomainOntologyAgent
   - PlainLanguageAgent
   - DocumentationAssemblerAgent
5. **Orchestrator** - Agent workflow management
6. **Context Manager** - Working vs long-term memory with compaction

### Key Workflows:
- Data ingestion and parsing
- Technical analysis with clarification requests
- Ontology mapping (OMOP, LOINC, SNOMED)
- Human-readable documentation generation
- Human-in-the-loop review and approval
- Context compaction for large sessions

### Next Steps:
To use this in Kaggle:
1. Add your Google Gemini API key as a Kaggle secret named 'GOOGLE_API_KEY'
2. Upload your data dictionary files
3. Run the notebook cells in order
4. Review and approve generated documentation
5. Export the final documentation

### Extending the System:
- Add custom agents for domain-specific processing
- Create new Toon types for your use case
- Implement additional ontology mappings
- Build a web UI using Streamlit or Gradio
- Add version control for documentation